In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/공모전/KTH/data/전처리전/IC
!ls -l

/content/drive/MyDrive/공모전/KTH/data/전처리전/IC
total 31
-rw------- 1 root root 28795 Jun 26 17:08 IC_정보.csv
-rw------- 1 root root  1391 Jun 27 06:25 서울교통공사_건물정보_20210228.csv
-rw------- 1 root root   742 Jun 27 07:28 차량기지_위치데이터.csv


In [6]:
import pandas as pd
import numpy as np

In [16]:
center = pd.read_csv('/content/drive/MyDrive/공모전/KTH/data/전처리후/IC/차량기지_위치데이터.csv')
ic = pd.read_csv('/content/drive/MyDrive/공모전/KTH/data/전처리후/IC/IC_위도_경도.csv')

In [23]:
center = center[['명칭', '위도', '경도']]
center.head(2)

,명칭,위도,경도
0,군자차량사업소,37.55380,127.05452
1,신정차량사업소,37.51167,126.86293


In [18]:
ic.head(2)

,IC명,위도,경도
0,안성IC,36.994219,127.156069
1,오산IC,37.142691,127.086371


In [19]:
center.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   명칭      10 non-null     object 
 1   소재지     10 non-null     object 
 2   위도      10 non-null     float64
 3   경도      10 non-null     float64
dtypes: float64(2), object(2)
memory usage: 448.0+ bytes


In [20]:
ic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   IC명     94 non-null     object 
 1   위도      94 non-null     float64
 2   경도      94 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.3+ KB


In [21]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from haversine import haversine

In [24]:
def cal_distance(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
  '''
  Input:
  df1: 차량 사업소(차량기지) -> 명칭, 위도, 경도
  df2: IC(inter connect) -> IC명, 위도, 경도

  Output:
  df: 차량 사업소-IC -> 차량 사업소 명칭, IC 명칭, 위도-경도에 대한 거리
  '''
  res = []
  for _, row in df1.iterrows():
    new_row = []
    new_row.append(row[0])
    for _, row2 in df2.iterrows():
      distance = haversine(row[1:3], row2[1:3], unit='km')
      res.append(new_row + [row2[0], distance])
  return pd.DataFrame(res, columns=['차량기지명', 'IC명', '거리(km)'])

In [25]:
highway_distance =\
  cal_distance(center, ic).sort_values('거리(km)').reset_index(drop=True)
highway_distance

,차량기지명,IC명,거리(km)
0,고덕차량사업소,강일IC,1.168301
1,고덕차량사업소,강일IC,1.259153
2,고덕차량사업소,상일IC,1.634431
3,고덕차량사업소,선동IC,1.672040
4,고덕차량사업소,상일IC,1.748786
...,...,...,...
935,천왕차량사업소,음성IC,87.728248
936,신정차량사업소,음성IC,88.717202
937,도봉차량사업소,음성IC,93.944064
938,방화차량사업소,음성IC,97.353536


In [46]:
highway_distance_mean =\
  highway_distance.groupby(['차량기지명', 'IC명']).mean().reset_index().sort_values('거리(km)')
highway_distance_mean.reset_index(drop=True, inplace=True)
highway_distance_mean

,차량기지명,IC명,거리(km)
0,고덕차량사업소,강일IC,1.213727
1,고덕차량사업소,선동IC,1.672040
2,고덕차량사업소,상일IC,1.691608
3,수서차량사업소,송파IC,2.580486
4,고덕차량사업소,토평IC,2.602116
...,...,...,...
785,천왕차량사업소,음성IC,87.728248
786,신정차량사업소,음성IC,88.717202
787,도봉차량사업소,음성IC,93.944064
788,방화차량사업소,음성IC,97.353536


In [49]:
highway_distance_mean['거리(km)'].max()

98.16767068138083

In [47]:
highway_distance_mean.to_csv('/content/drive/MyDrive/공모전/KTH/data/전처리후/IC/IC_차량기지_거리.csv',
                             index=False, encoding='utf-8')

In [48]:
center_names = highway_distance_mean['차량기지명'].unique().tolist()
center_names

['고덕차량사업소',
 '수서차량사업소',
 '신내차량사업소',
 '방화차량사업소',
 '창동차량사업소',
 '천왕차량사업소',
 '신정차량사업소',
 '지축차량사업소',
 '도봉차량사업소',
 '군자차량사업소']

In [59]:
a = []
x = highway_distance_mean[highway_distance_mean['차량기지명'] == '고덕차량사업소']
a.append(x[x['거리(km)'] == x['거리(km)'].min(axis=0)])
a.append(x[x['거리(km)'] == x['거리(km)'].min(axis=0)])
pd.concat(a, axis=0)

,차량기지명,IC명,거리(km)
0,고덕차량사업소,강일IC,1.213727
0,고덕차량사업소,강일IC,1.213727


In [61]:
min_distance = []
for name in center_names:
  dist = 1000
  tmp_df = highway_distance_mean[highway_distance_mean['차량기지명'] == name]
  row = tmp_df[tmp_df['거리(km)'] == tmp_df['거리(km)'].min(axis=0)]
  min_distance.append(row)
min_dist_highway_distance = pd.concat(min_distance, axis=0)
min_dist_highway_distance

,차량기지명,IC명,거리(km)
0,고덕차량사업소,강일IC,1.213727
3,수서차량사업소,송파IC,2.580486
6,신내차량사업소,구리IC,3.321785
7,방화차량사업소,김포IC,3.465914
10,창동차량사업소,별내IC,4.180827
13,천왕차량사업소,금천IC,4.564908
24,신정차량사업소,금천IC,6.159419
29,지축차량사업소,일산IC,7.346412
35,도봉차량사업소,별내IC,7.696694
47,군자차량사업소,토평IC,9.667816


In [62]:
min_dist_highway_distance.to_csv('/content/drive/MyDrive/공모전/KTH/data/전처리후/IC/IC_차량기지_최소거리.csv',
                                 index=False, encoding='utf-8')